<h1>1. 트위터 데이터 크롤링</h1>

In [47]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import time
import json
from datetime import timedelta, date

In [48]:
start_time = time.time()

#빈 리스트 생성
tweets_list = []

twitter_scrape = sntwitter.TwitterSearchScraper("#아이브 since:2021-12-01 until:2022-01-09 lang:ko")

#날짜와 해시태그 수집
for i, tweet in enumerate(twitter_scrape.get_items()):
    tweets_list.append([tweet.date, tweet.hashtags])

#데이터 프레임 생성
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime','hashtag'])

end_time = time.time()
print(f"Time is {end_time - start_time} sec")

Time is 797.0616579055786 sec


In [49]:
tweets_df

,Datetime,hashtag
0,2022-01-08 23:56:18+00:00,"[아이브, 장원영, IVE, JANGWONYOUNG]"
1,2022-01-08 23:52:56+00:00,"[아이브, 장원영, IVE, JANGWONYOUNG]"
2,2022-01-08 23:52:55+00:00,"[아이브, 장원영, IVE, JANGWONYOUNG]"
3,2022-01-08 23:52:52+00:00,"[아이브, 장원영, IVE, JANGWONYOUNG]"
4,2022-01-08 23:52:50+00:00,"[아이브, 장원영, IVE, JANGWONYOUNG]"
...,...,...
13716,2021-12-01 00:51:44+00:00,"[IVE, 아이브]"
13717,2021-12-01 00:38:47+00:00,"[아이브, IVE]"
13718,2021-12-01 00:22:04+00:00,"[아이브, IVE, 안유진, 가을, 레이, 장원영, 리즈, 이서, ELEVEN, 일..."
13719,2021-12-01 00:12:39+00:00,"[아이브, ELEVEN]"


In [7]:
tweets_df.to_csv('/users/noah/desktop/hashtag_ive_.csv', encoding="utf-8-sig")

<h1>2. 기본 데이터 전처리</h1>

In [50]:
import pandas as pd
import ast
import numpy as np
import time

In [51]:
ht = pd.read_csv("/users/noah/desktop/hashtag_ive_.csv")

In [52]:
#데이터 프레임으로 변환
ht_ = pd.DataFrame(ht)

In [53]:
#불필요한 열 제거
ht = ht_.drop(['Unnamed: 0', 'Datetime'], axis = 'columns')
ht

,hashtag
0,"['아이브', '장원영', 'IVE', 'JANGWONYOUNG']"
1,"['아이브', '장원영', 'IVE', 'JANGWONYOUNG']"
2,"['아이브', '장원영', 'IVE', 'JANGWONYOUNG']"
3,"['아이브', '장원영', 'IVE', 'JANGWONYOUNG']"
4,"['아이브', '장원영', 'IVE', 'JANGWONYOUNG']"
...,...
13787,"['IVE', '아이브']"
13788,"['아이브', 'IVE']"
13789,"['아이브', 'IVE', '안유진', '가을', '레이', '장원영', '리즈',..."
13790,"['아이브', 'ELEVEN']"


<h2> 노드 만들기 </h2>

In [54]:
#복사본 만들기
ht_node = ht.copy()

In [55]:
#리스트로 변환
for i in range(len(ht_node)):
    ht_node['hashtag'][i] = ast.literal_eval(ht['hashtag'][i])

In [56]:
#노드 list 만들기
node_ = []

for i in range(len(ht_node)):
    node_ += ht_node['hashtag'][i]

In [165]:
#노드 list -> set -> list 로 중복제거
node_set = set(node_)
node = list(node_set)
node_data = pd.DataFrame(node)
print(node_data)

             0
0         미녀월드
1         겨울코디
2      프라이빗메시지
3          엔시티
4        캘리그라피
...        ...
1648   아이브뮤직뱅크
1649        바자
1650  우리가사랑한노래
1651        빌리
1652      음악방송

[1653 rows x 1 columns]


In [172]:
node_data.rename(columns = {0: 'Label'}, inplace = True)
print(node_data)

         Label
0         미녀월드
1         겨울코디
2      프라이빗메시지
3          엔시티
4        캘리그라피
...        ...
1648   아이브뮤직뱅크
1649        바자
1650  우리가사랑한노래
1651        빌리
1652      음악방송

[1653 rows x 1 columns]


In [ ]:
#노드 csv로 저장
#node_data.to_csv("/users/noah/desktop/hashtag_node.csv", header = True, encoding = "utf-8-sig")

<h2> 상위 노출 노드 선택 </h2>

In [145]:
nodes = pd.read_csv("/users/noah/desktop/hashtag_node.csv")

In [146]:
print(nodes)

        Id       Label
0        0        포토타임
1        1        IDOL
2        2        blip
3        3    이런귀요미들ㅜㅜ
4        4         신기루
...    ...         ...
1648  1648     fansign
1649  1649         유태양
1650  1650  KIMJAEHWAN
1651  1651      VETEZE
1652  1652        퍼플키스

[1653 rows x 2 columns]


In [129]:
#빈 카운트 리스트 만들기
counts = np.zeros(len(nodes['Label']))

In [132]:
#노드별 노출 카운트 세기
#counts 세기
for i in range(len(nodes['Label'])):
    for j in range(len(ht_node)):
        if nodes['Label'][i] in ht_node['hashtag'][j]:
            counts[i] += 1

In [148]:
#데이터프레임에 카운트 추가
nodes['counts'] = counts

In [149]:
#새 id 넘버
a = []
for i in range(37):
    a.append(i) 

In [151]:
#count 50 미만 제거, id 넘버 다시 붙이기, 데이터프레임 구조 변경
nodes_new = nodes[nodes['counts'] >= 140]
print(len(nodes_new))
nodes_copy = nodes_new.copy()
nodes_copy['id'] = a
nodes_copy.drop(columns = {'Id'}, inplace = True)
nodes_copy.rename(columns = {'id' : 'Id'}, inplace = True)
nodes_final = nodes_copy.reindex(columns = ['Id', 'Label', 'counts'])
print(nodes_final)

37
      Id           Label   counts
74     0            대리찍사    167.0
200    1      IVE_ELEVEN   1898.0
238    2             아이브  13792.0
264    3           ウォニョン    467.0
322    4        Wonyoung    207.0
326    5          LEESEO   1157.0
363    6              이서   1787.0
378    7         ANYUJIN    232.0
442    8             アイヴ    612.0
478    9             LIZ    917.0
489   10              유진   1557.0
496   11              퇴근    156.0
522   12              출근    155.0
623   13              レイ    877.0
812   14             Rei    173.0
914   15             장원영   1606.0
939   16              원영   2362.0
961   17    JangWonyoung    179.0
989   18         AnYujin    458.0
1027  19             REI   1473.0
1067  20          ELEVEN   2700.0
1073  21              가을   1553.0
1099  22              리즈   1631.0
1113  23             IVE  11278.0
1164  24           YUJIN    631.0
1173  25             최파타    143.0
1191  26  아이브와_함께_heaven    382.0
1213  27              레이   2298.0
1232  28   

In [152]:
nodes_new.to_csv("/users/noah/desktop/hashtag_node_new37.csv", header = True, encoding = "utf-8-sig", index = False)

<h2> 상위 노드에 대한 엣지 생성 </h2>

In [153]:
from itertools import combinations
import numpy as np
import time

In [154]:
nodes = pd.read_csv("/users/noah/desktop/hashtag_node_new37.csv")

In [155]:
#복사본 생성
df = ht_node.copy()

In [156]:
#기본 프레임 만들기
edge_base = pd.DataFrame(list(combinations(nodes['Label'], 2)))
edge_base.rename(columns = {0:'source', 1:'target'}, inplace = True)
weight = np.zeros(len(edge_base))
print(edge_base)

           source        target
0            대리찍사    IVE_ELEVEN
1            대리찍사           아이브
2            대리찍사         ウォニョン
3            대리찍사      Wonyoung
4            대리찍사        LEESEO
..            ...           ...
661      WONYOUNG  JANGWONYOUNG
662      WONYOUNG           안유진
663    아이브고_일레븐교시  JANGWONYOUNG
664    아이브고_일레븐교시           안유진
665  JANGWONYOUNG           안유진

[666 rows x 2 columns]


In [157]:
#test 해보기
edge_test = edge_base.copy()
edge_list = edge_test.values.tolist()
print(edge_list)

[['대리찍사', 'IVE_ELEVEN'], ['대리찍사', '아이브'], ['대리찍사', 'ウォニョン'], ['대리찍사', 'Wonyoung'], ['대리찍사', 'LEESEO'], ['대리찍사', '이서'], ['대리찍사', 'ANYUJIN'], ['대리찍사', 'アイヴ'], ['대리찍사', 'LIZ'], ['대리찍사', '유진'], ['대리찍사', '퇴근'], ['대리찍사', '출근'], ['대리찍사', 'レイ'], ['대리찍사', 'Rei'], ['대리찍사', '장원영'], ['대리찍사', '원영'], ['대리찍사', 'JangWonyoung'], ['대리찍사', 'AnYujin'], ['대리찍사', 'REI'], ['대리찍사', 'ELEVEN'], ['대리찍사', '가을'], ['대리찍사', '리즈'], ['대리찍사', 'IVE'], ['대리찍사', 'YUJIN'], ['대리찍사', '최파타'], ['대리찍사', '아이브와_함께_heaven'], ['대리찍사', '레이'], ['대리찍사', '데이터'], ['대리찍사', '프리뷰'], ['대리찍사', 'Yujin'], ['대리찍사', 'GAEUL'], ['대리찍사', 'ive'], ['대리찍사', 'WONYOUNG'], ['대리찍사', '아이브고_일레븐교시'], ['대리찍사', 'JANGWONYOUNG'], ['대리찍사', '안유진'], ['IVE_ELEVEN', '아이브'], ['IVE_ELEVEN', 'ウォニョン'], ['IVE_ELEVEN', 'Wonyoung'], ['IVE_ELEVEN', 'LEESEO'], ['IVE_ELEVEN', '이서'], ['IVE_ELEVEN', 'ANYUJIN'], ['IVE_ELEVEN', 'アイヴ'], ['IVE_ELEVEN', 'LIZ'], ['IVE_ELEVEN', '유진'], ['IVE_ELEVEN', '퇴근'], ['IVE_ELEVEN', '출근'], ['IVE_ELEVEN', 'レイ'], ['IVE_ELEVEN', 'Rei'], ['IVE_ELEVEN'

In [158]:
# weight 찾기
start_time = time.time()

for i in range(len(edge_list)):
    for j in range(len(df)):
        if (edge_list[i][0] in df['hashtag'][j]) & (edge_list[i][1] in df['hashtag'][j]):
            weight[i] += 1
            
#for [a, b], if both 'a' and 'b' are in [a, b, c, d] then plus the weight
            
end_time = time.time()
print(f"Time is {end_time - start_time} sec")

Time is 63.92641806602478 sec


In [175]:
#weight 분석해보기
print(list(map(int, weight))[0:10])

weight_df = pd.DataFrame(weight)
weight_df.value_counts()

[0, 167, 0, 0, 10, 69, 0, 0, 9, 64]


0.0        198
1.0         39
2.0         20
3.0         15
12.0        13
          ... 
212.0        1
214.0        1
217.0        1
219.0        1
11278.0      1
Length: 227, dtype: int64

In [160]:
edge_base['weight'] = weight
print(edge_base)

           source        target  weight
0            대리찍사    IVE_ELEVEN     0.0
1            대리찍사           아이브   167.0
2            대리찍사         ウォニョン     0.0
3            대리찍사      Wonyoung     0.0
4            대리찍사        LEESEO    10.0
..            ...           ...     ...
661      WONYOUNG  JANGWONYOUNG    72.0
662      WONYOUNG           안유진    41.0
663    아이브고_일레븐교시  JANGWONYOUNG     1.0
664    아이브고_일레븐교시           안유진     8.0
665  JANGWONYOUNG           안유진    10.0

[666 rows x 3 columns]


In [161]:
#이름 대신에 id에 weight 붙여서 양식 만들기
edge_id = pd.DataFrame(list(combinations(nodes['Id'], 2)))
edge_id.rename(columns = {0:'source', 1:'target'}, inplace = True)
edge_id['weight'] = weight
print(edge_id)

     source  target  weight
0        74     200     0.0
1        74     238   167.0
2        74     264     0.0
3        74     322     0.0
4        74     326    10.0
..      ...     ...     ...
661    1390    1569    72.0
662    1390    1575    41.0
663    1472    1569     1.0
664    1472    1575     8.0
665    1569    1575    10.0

[666 rows x 3 columns]


In [39]:
edge_id.to_csv("/users/noah/desktop/hashtag_edge_new37.csv", header = True, encoding = "utf-8-sig")